In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [2]:
#using the SQLite Table to read data.
con=sqlite3.connect(r'E:\AAIC\Module 3. Foundation of NLP and ML\Real world problem Predict rating given product reviews on Amazon\amazon-fine-food-reviews\database.sqlite')
#filtering only positive and negative reviews i,e 
#not taking into consideration those reviews with scores=3
filtered_data=pd.read_sql_query('''select * from reviews where score !=3''',con)


#Give review with score>3 a positive rating and review with a score <3 
def partitioned(x):
    if x>3:
        return 'Positive'
    else:
        return 'Negative'
#chenging the review
actualscore=filtered_data['Score']
Positive_Negative=actualscore.map(partitioned)
filtered_data['Score']=Positive_Negative



In [3]:

filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,Positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,Negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,Positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,Negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,Positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
filtered_data.shape

(525814, 10)

In [5]:
#EDA
#Data cleaning and removing duplication
display=pd.read_sql_query('select * from reviews where score !=3 and userid="AR5J8UI46CURR" order by ProductID',con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [6]:
#in the above reviews we can see the userid is same Profile name is same, helpingNumerator helpingDenominator score time summary text 
#all are same so how is it possilble, actuallu it is not duplicate but the same product with different flavours
#https://www.amazon.com/Clarks-Tilden-Cap-Oxford-Leather/dp/B000HDL1RQ/ref=lp_18637582011_1_1?srs=18637582011&ie=UTF8&qid=1562952894&sr=8-1&th=1
#https://www.amazon.com/Clarks-Tilden-Cap-Oxford-Leather/dp/B000HDOPYM/ref=lp_18637582011_1_1?srs=18637582011&ie=UTF8&qid=1562952894&sr=8-1&th=1

# we can consider these reviews as duplicates so remove these data

In [7]:
#sorting the data according to productid ascending order
sorted_data=filtered_data.sort_values('ProductId',axis=0,ascending=True)
sorted_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,Positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,Positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [8]:
#Duplication of entries
final=sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)
final.shape

(364173, 10)

In [9]:
#checking to see how much % of data still remains
(final['Id'].size)/(filtered_data['Id'].size)*100

69.25890143662969

It was seen that in two rows given below the value of helpfulnessNumerator is greater than HelpfulnessDenominator which is not 
Practically possible hence these two rows two are remoed from calculation (this might be some technical issues)

In [10]:
display=pd.read_sql_query('select * from reviews where score !=3 and Id=44737 or id=64422 order by ProductID',con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [11]:
#Because: HelpfulnessNumerator =persons says reviews is helpfull, HelpfulnessDenominator=Yes+No.

In [12]:
final=final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]
final.shape

(364171, 10)

In [13]:
#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

Positive    307061
Negative     57110
Name: Score, dtype: int64

so we have more positive points than the negative 

# Text Processing: Stemming, stopword removal and lemmatization. 

In [14]:
#if you go to the data and observe it, you will find that the most usefull features are the 'Summary' and 'Text'
#from the linear algebra we know that , given anything, if we convert it to the vector we can take the whole liverage of<br>
#Linear algebra

we have 5 techniques to convert the text (santences ) to vector.
<li>1.BoW</li>
<li>2.tf-idf</li>
<li>3.w2v</li>
<li>4.avg-w2v</li>
<li>5.avgw2v</li>

<li>Stemming: the words like-Testy, Testfull, tests represents the same meaning but across the reviews the customers use all
(there are two algo for stemming in nltk, 1.)porter stemmer 2.)Snawball Stemmer</li>
<li>Stopwords: The words like This, is, Not, Very, every etc are not that relevent to our analysis so it is stopwords</li>
<li>Lemmatization: in english there are the words which are breaked by space but they are single word eg.New York.</li>
<li>Sementic Meaning: the words like: testy, delicious, very testy are same in meaning but it unneccessarily creating the dimension of words(it is a drawback of BOW)</li>

# Text Preprocessing

In [15]:
import nltk

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop=set(stopwords.words('english')) #set of stopwords
sno=nltk.stem.SnowballStemmer('english')#initialising the snowball stemmer
def cleanhtml(sentence):#function to clean the word of any html-tags
    cleaner=re.compile('<.*?>')
    cleantext=re.sub(cleaner,' ',sentence)
    return cleantext
def cleanpunc(sentence):#function to clean the word of any punctuation 
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned
print(stop)
print('***************************************************')
print(sno.stem('testy'))


{'as', "mightn't", 'on', 're', "that'll", 'between', 's', 'hers', 'needn', 'below', 'being', 'haven', "shouldn't", 'can', 'those', 'shan', "aren't", 'ours', 'he', 'nor', 'very', 'or', 'won', 'itself', 'all', 'was', 'your', 'for', 'o', 'they', 'wouldn', "you'd", 'own', 'hasn', "hadn't", 'd', 'ma', 'yours', 'had', 'that', 'ourselves', "doesn't", 'which', 'will', 'not', 'if', 'her', 'shouldn', 'of', 'this', 'off', 'how', 'to', 'more', 'isn', "weren't", 'i', 'too', 'where', 'just', 'than', 'then', 'most', 'were', 'them', 'yourselves', "she's", 'himself', 'is', 'whom', 'through', 'my', 'it', 'any', 'who', 'their', 'when', 'myself', "don't", 'the', 'again', 'she', 'other', 'aren', 'have', 'with', 'about', 'over', 'into', 'here', 'should', 'theirs', 'did', 'themselves', "isn't", 'only', 'up', 'so', 'do', 'out', 'herself', 'an', 'and', 'doing', "won't", 'at', "couldn't", 'am', 'a', 'its', 'didn', "needn't", 'there', 'no', 'has', 'some', 'because', 't', 've', 'll', 'these', "wasn't", 'now', 'ab

# 1.)BoW (bag of words):

In [16]:
from sklearn import feature_extraction
count_vect=feature_extraction.text.CountVectorizer()
final_counts=count_vect.fit_transform(final['Text'].values)
type(final_counts)

scipy.sparse.csr.csr_matrix

In [17]:
final_counts.shape

(364171, 115281)

#from the shape:now we have a matrix of  364171 rows and 115281 columns.
#itmeans most of the values in a cell in any row is zero

In [18]:
#code for implementing step-by-step the check mentioned in the preprocessing
#
i=0
str1=' '
final_string=[]
all_positive_words=[]#store words from positive review here
all_negative_words=[]#stores word from -ve reviews here
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent)
    sent=cleanhtml(sent)#removes HTML tag
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i]=='positive':
                        all_positive_words.append(s)#list of all wordsuse
                    if (final['Score'].values)[i]=='negative':
                        all_negative_words.append(s) #list of all used 
                else:
                    continue
            else:
                continue
    #print(filtered_sentence)
    str1=b' '.join(filtered_sentence)#final string of cleaned words
    #print('**********************************************************')
    final_string.append(str1)
    i+=1

In [19]:
final['CleanedText']=final_string#adding a column of CleanedText which 

In [20]:
final['CleanedText'].head(3)

138706    b'witti littl book make son laugh loud recit c...
138688    b'grew read sendak book watch realli rosi movi...
138689    b'fun way children learn month year learn poem...
Name: CleanedText, dtype: object

In [21]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,Positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,Positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,Positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...


In [22]:
#stores final table into an SQlite table for future
conn=sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory=str
final.to_sql("Reviews",conn, schema=None, if_exists='replace')

# Bi-Grams and n-Grams.

Motivation: Now that we have our list of words describing positive and negative reviews lets analyse them.
    we began analysis by getting the frequency distribution of the words as shown below

In [23]:
frequency_distribution_positive=nltk.FreqDist(all_positive_words)
frequency_distribution_negative=nltk.FreqDist(all_negative_words)
print("Most Common Positive Words :",frequency_distribution_positive.most_common(20))
print('Most common Negative Words :',frequency_distribution_negative.most_common(20))


Most Common Positive Words : []
Most common Negative Words : []


In [24]:
#bi-gram, tri-gram and n-gram
#removing stop words like 'not'should be avoided before building n-gram 
count_vect=feature_extraction.text.CountVectorizer(ngram_range=(1,2))  #(1,2) represent we get unigram and bi-gram 
final_bigram_counts=count_vect.fit_transform(final['Text'].values)


In [25]:
final_bigram_counts.get_shape()

(364171, 2910192)

Here dimensionality increased drastically

# TF-IDF

In [26]:
tf_idf_vect=feature_extraction.text.TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(final['Text'].values)

In [27]:
final_bigram_counts.get_shape()

(364171, 2910192)

In [28]:
features=tf_idf_vect.get_feature_names()
len(features)

2910192

In [29]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

Notice above you can see it has both unigram and bigrams

In [30]:
#convert a row in saprsematix to a numpy array
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [31]:
def top_tfidf_feats(row,features, top_n=25):
    '''get top tfidf values in row and return them with their corresponding rank'''
    topn_ids=np.argsort(row)[::-1][:top_n]
    top_feats=[(features[i],row[i]) for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['features','tfidf']
    return df
top_tfidf=top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [32]:
top_tfidf

,features,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074
